<a href="https://colab.research.google.com/github/sorayutmild/Unsupervised-Thai-Document-Clustering-with-Sanook-news/blob/main/Document_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# จัดกลุ่มบทความ โดยใช้ unsupervised learning model
## Fisrt method, Hard clustering
### One document can be only one cluster
### 1. Text representation (Bag-of-Word, Doc2vec, Transformer embedding)
### 2. Clustering model (e.g., k-means clustering, Hierarchical clustering)

\
## Second method, Soft clustering (not cover)
### One document can be multiple clusters.
### Topic modeling (LDA)


# Topic
* Download dataset
* Text preprocessing
* Text representation
    * Bag-of-Words
    * Transformer Embedding
    * Clustering model
* Evaluation
* Error analysis

In [ ]:
!nvidia-smi

In [ ]:
!pip -q install pythainlp==3.0.0
!pip -q install python-crfsuite

In [ ]:
import pandas as pd
import numpy as np
import glob
from pythainlp import word_tokenize
from pythainlp.corpus import thai_stopwords
import re
import os
from tqdm.auto import tqdm

In [ ]:
text_file_folder = 'text_files'

## Download dataset

In [ ]:
# https://drive.google.com/file/d/16IhVdoLuFso28TVIpYoIogco_LZg1e17/view?usp=sharing
!gdown --id 16IhVdoLuFso28TVIpYoIogco_LZg1e17

# https://drive.google.com/file/d/1chetbLnMLRIqt0U8m3JMzO0xguGUEHX6/view?usp=sharing
!gdown --id 1chetbLnMLRIqt0U8m3JMzO0xguGUEHX6

!unzip -q ./text_files.zip

In [ ]:
df = pd.read_csv('sanook_news_all.csv')
display(df.head())
df.info()

In [ ]:
df['Label'].value_counts()

remove duplicate link rows

In [ ]:
df = df.drop_duplicates('Link').reset_index()
df

## Text preprocessing

In [ ]:
def read_text_file(text_path):
    with open(text_path) as f:
        texts = [line.strip() for line in f.readlines()]
    return texts

def get_raw_text(text_list:list):
    '''
    text_list : list of text paragraphs
    return : raw text
    '''
    return ' '.join(text_list)

# def get_list_text(text):
#     '''
#     get list text, separate with paragraph
#     '''
#     return text.split('\n')

def include_title(title, text_list):
    return [title] + text_list

def tokenize(text):
    tokenized_text = [word for word in word_tokenize(text) if word and not re.search(pattern=r"\s+", string=word)] # segment and not ' word '
    return tokenized_text

def replace_url(text):
    URL_PATTERN = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""
    return re.sub(URL_PATTERN, 'xxurl', text)

def replace_special_char(text):
    special_char = r"""[~@&#<>,}{()*:;"'-+=_?"\"/$%^%$!ๆ‘’“”…่่่่่่่่่ạễ•′–0-9]"""
    text = re.sub(special_char, '', text)
    text = text.replace("[", '').replace("]", '').replace('-', '')
    return text

def clean_text(text):
    '''
    1. link
    2. symbols, numbers, special characters
    '''
    #pre rules
    text = text.lower().strip()
    text = replace_url(text)
    text = replace_special_char(text)
    cleaned_text = text

    return cleaned_text


# read_text_file('text_files/ข่าวไอที_1647.txt')
# test_text = '[11-กุมภาพันธ์-2556] ไอบัดดี้โฟนเรา..วันนี้ มีบทความดีๆมาฝาก เกี่ยวกับมารยาทการใช้มือถือที่เราต้องใส่ใจสังคมคนรอบๆข้างตัวเรา (บ้าง)...โทรศัพท์มือถือในยุคปัจจุบันกลายเป็นปัจจัยที่4 ของคนเราไปเสียแล้ว ไม่ว่าเราจะนั่งชิลๆ อยู่กับบ้าน หรือแม้กระทั่งต้องออกนอกบ้านก็ไม่สามารถอยู่ห่างมือถือได้เลย'
# process_text = tokenize(test_text)

In [ ]:
df['File_path'] = [os.path.join(text_file_folder, fn) for fn in df['File_name'].values]
df['list_text'] = df['File_path'].map(read_text_file)
df['list_text'] = [include_title(t, lt) for t, lt in zip(df['Title'].values, df['list_text'].values)]
df['raw_text'] = df['list_text'].map(get_raw_text)
df['cleaned_text'] = df['raw_text'].map(clean_text)
# df['list_clean_text'] = df['cleaned_text'].map(get_list_text)
df.head()

In [ ]:
# clear empty document (only have title)
df = df[df["raw_text"]!=df["Title"]]
df.reset_index(inplace=True, drop=True)
df.shape

In [ ]:
df['cleaned_text'][100]

## Hard clustering

### Text representation

#### Bag-of-Word

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer=tokenize, min_df=100, sublinear_tf=True, )
tfidf_fit = tfidf.fit(df['cleaned_text'])
text_vector = tfidf_fit.transform(df['cleaned_text'])
text_vector.shape # (14821, 7613) unclean # (14821, 4102) clean

In [ ]:
tfidf_fit.get_feature_names_out()

In [ ]:
word = pd.DataFrame({
    'word':tfidf_fit.get_feature_names_out()
})

word.to_csv('word.csv', index=False, encoding='utf-8-sig')

In [ ]:
text_vector.toarray()

In [ ]:
np.save('tfidf_vectors', text_vector, allow_pickle=True)

#### Transformer embedding

In [ ]:
!pip -q install sentence_transformers==2.1.0

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import models
from pythainlp.tokenize import sent_tokenize

In [335]:
model_name = 'mrp/simcse-model-wangchanberta' # 'sorayutmild/simcse-model-wangchanberta-finetuned-sanook-news' # 'mrp/simcse-model-wangchanberta' # monsoon-nlp/bert-base-thai' # 'xlm-roberta-base' # airesearch/wangchanberta-base-att-spm-uncased'
# word_embedding_model = models.Transformer(model_name, max_seq_length=256)
model = SentenceTransformer(model_name)

Downloading:   0%|          | 0.00/123 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/421M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/905k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/364 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.18M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/589 [00:00<?, ?B/s]

In [336]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 32, 'do_lower_case': False}) with Transformer model: CamembertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [ ]:
# pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode='cls')
# model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [337]:
# for test feature extraction

t1 = ['วันนี้อากาศดีมาก']
t2 = ['วันนี้อากาศดีมากที่สุดในรอบปี', 'อากาศดีจังเลยวันนี้', 'ท้องฟ้าแจ่มใสดีจัง', 'อยากกินข้าวขาหมูเนอะ']

a = model.encode(t1, normalize_embeddings=True)
b = model.encode(t2, normalize_embeddings=True)

# cosine similarity
np.inner(a, b)

array([[0.76279557, 0.7769643 , 0.7523302 , 0.6738297 ]], dtype=float32)

In [338]:
from pythainlp.tag import NER
ner = NER(engine="thainer")

In [339]:
def number_of_ner(text):
    text_ner = ner.tag(text, tag=True, pos=False)
    # print(text_ner)
    n_ner = len(re.findall('</', text_ner))
    return n_ner

text = "ทดสอบนายสรยุทธ มียิ้มได้ไปเที่ยวห้างดังย่านภูเก็ตกับนายกรัฐมนตรีของประเทศศรีลังกา"
number_of_ner(text)

3

In [340]:
def document_vectorize(sentence_vector, w):
    assert sentence_vector.shape[0] == w.shape[0]
    vd = np.sum(w * sentence_vector, axis=0) / np.sum(w)
    return vd

def weigth_one(number_of_sentences):
    return np.array([1]*number_of_sentences).reshape(number_of_sentences, 1)

def weigth_with_ner(text):
    # np_number_of_ner = np.vectorize(number_of_ner)
    w_ner = number_of_ner(text)+1
    return w_ner

def truncate_text(text, max_char):
    return text[:max_char]

In [341]:
def embedding_pipeline(text, min_char, max_char):
    # All sentences tokenize
    sentences = np.array(sent_tokenize(text, engine='crfcut'))
    # check sentence > min 
    np_len = np.vectorize(len)
    sentences = sentences[np_len(sentences)>min_char]
    # truncate if exceed max 
    np_truncate_text = np.vectorize(truncate_text)
    # sentences = np.array([truncate_text(s, max_char) for s in sentences])
    sentences = np_truncate_text(sentences, max_char)
    # All sentences encoding
    embedding = model.encode(sentences, normalize_embeddings=True)
    # Calculate document representation
    # w = weigth_one(embedding.shape[0])
    # w = np.array([weigth_with_ner(s) for s in sentences]).reshape(embedding.shape[0], 1)
    np_weigth_with_ner = np.vectorize(weigth_with_ner)
    w = np_weigth_with_ner(sentences).reshape(embedding.shape[0], 1)
    document_embedding = document_vectorize(embedding, w)
    return document_embedding

take time about 30 mins with gpu



In [ ]:
vector_representation = np.array([embedding_pipeline(text, min_char=10, max_char=512) for text in tqdm(df['cleaned_text'].values)])

  0%|          | 0/14821 [00:00<?, ?it/s]

In [ ]:
np.save('simcse_model_wangchanberta_finetuned_with_NER_weight_wa', vector_representation, allow_pickle=True)

### Clustering model

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
k = 10
km = KMeans(n_clusters=k, max_iter=100, n_init=55, )

In [ ]:
vector_representation = np.load('simcse_model_wangchanberta_with_NER_weight_wa.npy', allow_pickle=True)

In [ ]:
vector_representation.shape

In [ ]:
np.any(np.isnan(vector_representation))

In [ ]:
# vector_representation = np.nan_to_num(vector_representation)
# np.any(np.isnan(vector_representation))

In [ ]:
km.fit(vector_representation)

In [ ]:
# order_centroids = km.cluster_centers_.argsort()[:, ::-1]
# order_centroids

In [ ]:
# term = tfidf.get_feature_names_out()
# for i in range(k):
#     print(f'Cluster {i+1}: ', end='')
#     for ind in order_centroids[i, :10]:
#         print(term[ind], end=' ')
#     print()

## Evaluation

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

In [ ]:
km.labels_

In [ ]:
def map_predict_to_label(df, predict_vector):
    map = dict()
    df['Predict'] = km.labels_
    unique_class_predict = np.unique(predict_vector)
    for i in unique_class_predict:
        # find max label in each predict and select it as a class
        c = df[df['Predict']==i]['Label'].mode().values
        map[i+1] = c
        df.loc[df['Predict']==i, ['Predict']] = c
    return df, map

df, map = map_predict_to_label(df, km.labels_)

In [ ]:
map

unclean BoW 0.8146797717184527 \
clean BoW 0.8216044801295459 \
simcse_model_wangchanberta 0.8330072194858646 \
simcse_model_wangchanberta with wa (not add 1) 0.843667768706565 \
simcse_model_wangchanberta with wa 0.8445449025032049 \
simcse_model_wangchanberta with wa finetuned 0.7367923891775184 \

In [ ]:
f1_score(df['Label'], df['Predict'], average='micro')

In [ ]:
accuracy_score(df['Label'], df['Predict'])

## Error analysis

In [ ]:
error_df = df[df['Label']!=df['Predict']]
error_df.info()

In [ ]:
error_df['case'] = error_df['Label'] + ',' + error_df['Predict']

In [ ]:
error_df['case'].value_counts()[:10]

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [ ]:
idx = 15

display(error_df[['Label', 'Predict']].iloc[idx])
pp.pprint(error_df['raw_text'].iloc[idx])